Implementing Decision Tree and Random Forest from Scratch


In [30]:
#1
import numpy as np
import pandas as pd
df = pd.read_csv('/content/titanic.csv')

# Dropped redundant/unnecessary columns
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Handled missing values
df['Age'] = df['Age'].fillna(df['Age'].mean())  # Filled Age with mean value
df.dropna(subset=['Embarked'], inplace=True)      # Dropped the rows with missing value of Embarked

# Hot encoding for categorical columns
df = pd.get_dummies(df, columns=['Sex', 'Embarked'])
X = df.drop(['Survived'], axis=1).values.astype(float)
y = df['Survived'].values.astype(int)

def training_testing(X, y, testsize=0.2, randomstate=42):
    random_number_generate = np.random.RandomState(randomstate)
    index = np.arange(len(y))
    random_number_generate.shuffle(index)
    n_test = int(len(y) * testsize)
    testing_index = index[:n_test]
    training_index = index[n_test:]
    return X[training_index], X[testing_index], y[training_index], y[testing_index]
X_train_np, X_test_np, y_train_np, y_test_np = training_testing(X, y, testsize=0.2, randomstate=42)


class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.min_samples_split = 2
        self.tree_ = None           # will be holding the learned tree dictonary
        self.max_features = None

    def gini_impurity(self, y):
        if len(y) == 0:
            return 0.0
        category, counts = np.unique(y, return_counts=True)
        probability = counts / counts.sum()
        return 1.0 - np.sum(probability ** 2)

    def information_gain(self, y_parent, y_left, y_right, parent=None):
        if parent is None:
            parent = self.gini_impurity(y_parent)
        n = len(y_parent)
        n_l, n_r = len(y_left), len(y_right)
        if n_l == 0 or n_r == 0:
            return 0.0
        weighted = (n_l / n) * self.gini_impurity(y_left) + (n_r / n) * self.gini_impurity(y_right)
        return parent - weighted


    def threshold_candidate_determine(self, values):
        uniq_val = np.unique(values)
        if len(uniq_val) <= 1:
            return []
        uniq_val.sort()
        return (uniq_val[:-1] + uniq_val[1:]) / 2.0

    def best_spliting_finding(self, X, y, feature_index=None):
        n_samples, n_features = X.shape
        if feature_index is None:
            feature_index = np.arange(n_features)
        best_gain = 0.0
        best_feature = None
        best_threshold = None
        parent = self.gini_impurity(y)

        for feature in feature_index:
            col = X[:, feature]
            threshold = self.threshold_candidate_determine(col)
            for thres in threshold:
                left = col <= thres
                right = ~left
                if left.sum() == 0 or right.sum() == 0:
                    continue
                gain = self.information_gain(y, y[left], y[right], parent)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = thres
        return best_feature, best_threshold

    def majority_class(self, y):
        values, counts = np.unique(y, return_counts=True)
        return int(values[np.argmax(counts)])

    def fit(self, X, y, depth=0):
        if len(np.unique(y)) == 1:
            leaf = {'type': 'leaf', 'class': int(y[0])}
            if depth == 0:
                self.tree_ = leaf
            return leaf

        if (self.max_depth is not None and depth >= self.max_depth) or (len(y) < self.min_samples_split):
            leaf = {'type': 'leaf', 'class': self.majority_class(y)}
            if depth == 0:
                self.tree_ = leaf
            return leaf

        n_features = X.shape[1]
        if self.max_features is None or self.max_features >= n_features:
            feature_index = None  # using all features
        else:
            feature_index = np.random.choice(n_features, self.max_features, replace=False)

        # Finding best split
        best_feature, best_threshold = self.best_spliting_finding(X, y, feature_index)

        if best_feature is None:
            leaf = {'type': 'leaf', 'class': self.majority_class(y)}
            if depth == 0:
                self.tree_ = leaf
            return leaf

        # Partitioning the data
        left = X[:, best_feature] <= best_threshold
        right = ~left

        # Recursively building children
        left_child = self.fit(X[left], y[left], depth + 1)
        right_child = self.fit(X[right], y[right], depth + 1)

        node = {
            'type': 'node',
            'feat': int(best_feature),
            'threshold': float(best_threshold),
            'left': left_child,
            'right': right_child
        }
        if depth == 0:
            self.tree_ = node
        return node

    def prediction(self, tree, instance):
        node = tree
        while node['type'] != 'leaf':
            feature = node['feat']
            thres = node['threshold']
            if instance[feature] <= thres:
                node = node['left']
            else:
                node = node['right']
        return node['class']

    def prediction_whole(self, X):
        return np.array([self.prediction(self.tree_, row) for row in X])


# ------------------ Train/Evaluate DecisionTree ------------------
decision_tree = DecisionTree(max_depth=5)
final_tree = decision_tree.fit(X_train_np, y_train_np)  # returns the tree structure

# predictionions on the test set
y_pred_dt_scr = np.array([decision_tree.prediction(final_tree, row) for row in X_test_np])

# Calculate accuracy manually: correctly predictioned / total
correct = np.sum(y_pred_dt_scr == y_test_np)
accuracy_dt_scr = correct / len(y_test_np)

print("Decision Tree Classifier (from scratch) Accuracy:", accuracy_dt_scr)


Decision Tree Classifier (from scratch) Accuracy: 0.8192090395480226


In [31]:
#2
import numpy as np

class RandomForest:
    def __init__(self, n_estimate=100, max_depth=None):
        self.n_estimate = n_estimate
        self.max_depth = max_depth
        self.trees_ = []

    def fit(self, X, y):
        self.trees_.clear()
        n_features = X.shape[1]
        # Used sqrt(d) features at each split
        max_feature = int(np.sqrt(n_features))
        if max_feature < 1:
            max_feature = 1

        for _ in range(self.n_estimate):
            X_sample, y_sample = self.bootstrap_sampling(X, y)
            tree = DecisionTree(max_depth=self.max_depth)
            tree.max_feature = max_feature
            tree.fit(X_sample, y_sample)
            self.trees_.append(tree)

    def bootstrap_sampling(self, X, y):
        n = len(y)
        idx = np.random.randint(0, n, size=n)
        return X[idx], y[idx]

    def predict(self, X):
        # Collected predictions from each tree; shape (n_estimate, n_samples)
        predictions = np.vstack([tree.prediction_whole(X) for tree in self.trees_])
        predicted = []
        for i in range(predictions.shape[1]):
            values, counts = np.unique(predictions[:, i], return_counts=True)
            predicted.append(values[np.argmax(counts)])
        return np.array(predicted)


# ------------------ Train/Evaluate RandomForest ------------------
random_forest = RandomForest(n_estimate=25, max_depth=5)
random_forest.fit(X_train_np, y_train_np)

# Predictions on the test set
y_pred_rf_scr = random_forest.predict(X_test_np)

# Calculate accuracy manually
correct_rf = np.sum(y_pred_rf_scr == y_test_np)
accuracy_rf_scr = correct_rf / len(y_test_np)

print("Random Forest Classifier (from scratch) Accuracy:", accuracy_rf_scr)


Random Forest Classifier (from scratch) Accuracy: 0.8305084745762712
